Variáveis Objetivo (Soja, milho, trigo e algodão)

In [68]:
# Importing the required libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [69]:
class Crop:
    def __init__(self, id, name, link, file_name, comm_file):
        self.id = id
        self.name = name
        self.link = link
        self.file_name = file_name
        self.comm_file = comm_file

    def web_scrap(self):
        data = requests.get(self.link).text
        
        # Creating BeautifulSoup object
        soup = BeautifulSoup(data, 'html.parser')

        # Creating list with all tables
        tables = soup.find_all('table')

        #  Looking for the table with the classes 'wikitable' and 'sortable'
        table = soup.find('table', class_='table table-main orange')

        # Defining of the dataframe
        df = pd.DataFrame(columns=['Data', 'Estadual'])

        dates = []
        states = []
        countrys = []
        # Collecting Ddata
        for row in table.tbody.find_all('tr'):    
            # Find all data for each column
            head = row.find_all('th')
            columns = row.find_all('td')
            
            if(head != []):
                date = head[0].text.strip()
                dates.append(date)

            
            if(columns !=[]):
                value1 = columns[0].text.strip()
                value2 = columns[1].text.strip()

                states.append(value1)
                countrys.append(value2)

        df = pd.DataFrame(list(zip(dates, states, countrys)),columns =['Data', 'Estadual', 'País'])
        df['Id'] = self.id
        df['Cultivo'] = self.name

        df.to_csv(self.file_name, sep=';', index=False)

    def add_comm(self):
        df_comm = pd.read_csv(self.comm_file, sep=',')
        df_cult = pd.read_csv(self.file_name, sep=';')

        df_cult['Data'] = df_cult['Data'].astype('datetime64[ns]')
        df_cult = df_cult.replace(',', '.', regex=True)

        df_comm['Data'] = df_comm['Data'].astype('datetime64[ns]')
        df_comm = df_comm.replace(',', '', regex=True)
        df_comm.drop(columns=['Data','Vol.', "Var%"], inplace=True)

        df_cult['Estadual'] = df_cult['Estadual'].astype(float)
        df_cult['País'] = df_cult['País'].astype(float)
        df_cult['Cultivo'] = df_cult['Cultivo'].astype('string')

        df_comm['Último'] = df_comm['Último'].astype(float)
        df_comm['Abertura'] = df_comm['Abertura'].astype(float)
        df_comm['Máxima'] = df_comm['Máxima'].astype(float)
        df_comm['Mínima'] = df_comm['Mínima'].astype(float)

        df = pd.concat([df_cult.reset_index(drop=True), df_comm.reset_index(drop=True)], axis=1)
        df = df[['Data', 'Cultivo', 'Estadual', 'País', 'Id', 'Último', 'Abertura', 'Máxima', 'Mínima']]
        df.to_csv(self.file_name, sep=";", index=False)


In [70]:

path = "data library/"
path_comm = "data library/Commodities/"

soja = Crop(0, "Soja", "https://www.agrolink.com.br/cotacoes/historico/mt/soja-em-grao-sc-60kg", 
            path+"Soja_v1.csv", path_comm+"Soja-Chicago.csv")
milho = Crop(1, "Milho", "https://www.agrolink.com.br/cotacoes/historico/pr/milho-seco-sc-60kg", 
            path+"Milho_v1.csv", path_comm+"Milho-Chicago.csv")
trigo = Crop(2, "Trigo", "https://www.agrolink.com.br/cotacoes/historico/pr/trigo-em-grao-nacional-sc-60kg", 
             path+"Trigo_v1.csv", path_comm+"Trigo-Chicago.csv")
algodao = Crop(3, "Algodão", "https://www.agrolink.com.br/cotacoes/historico/mt/algodao-em-pluma-15kg", 
               path+"Algodao_v1.csv", path_comm+"Algodao-NY.csv")

soja.web_scrap()
milho.web_scrap()
trigo.web_scrap()
algodao.web_scrap()

soja.add_comm()
milho.add_comm()
trigo.add_comm()
algodao.add_comm()

Bolsa de Chicago

In [71]:
url = "https://br.investing.com/commodities/us-wheat-historical-data"